<a href="https://colab.research.google.com/github/swarathesh/Spark_ML/blob/master/Spark_Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0
```



In [8]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [2]:
spark = SparkSession.builder.appName('WallmartCustomer').getOrCreate()

/home/swarathesh60/.local/lib/python2.7/site-packages/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [4]:
df = spark.read.csv('Ecommerce_Customers.csv',header=True,inferSchema=True)

In [5]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
vectorAssembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],
  outputCol='features')

In [13]:
output = vectorAssembler.transform(df)

In [16]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [17]:
data = output.select('features','Yearly Amount Spent')

In [18]:
data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [19]:
train, test = data.randomSplit([0.8,0.2])

In [48]:
model = LinearRegression(labelCol='Yearly Amount Spent')

In [49]:
model_lr = model.fit(train)

In [50]:
predicitons = model_lr.evaluate(test)

In [51]:
predicitons.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-12.718883108004206|
|  5.244778639386141|
|-14.157163827210923|
|-23.208125811204468|
| 0.2131184725390085|
| -5.825818464625172|
|-3.8200969485631617|
|  5.966320706031581|
| -1.576327762832193|
| -2.020917744043288|
|  1.228955328024142|
| -5.310806846290063|
|-18.865437902006363|
| -9.861665535606733|
| -6.066180789537555|
|-2.3650661790026675|
|  7.924868452077476|
| 3.7538746266887983|
| 0.2750357493980573|
|  4.599942546570787|
+-------------------+
only showing top 20 rows



In [52]:
predicitons.r2

0.9828822879546597

# what if there are only features

In [53]:
output_features = test.select('features')

In [56]:
model_lr.transform(output_features).show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 332.6477529111978|
|[30.9716756438877...| 489.3938311175066|
|[31.0662181616375...| 463.0904570348853|
|[31.1239743499119...|510.15517965097024|
|[31.3895854806643...| 409.8564925874439|
|[31.5147378578019...| 495.6383064610866|
|[31.5171218025062...| 279.7385175989489|
|[31.6548096756927...|469.29710302151693|
|[31.6610498227460...|417.93468134273303|
|[31.7216523605090...|349.79784437591593|
|[31.7366356860502...| 495.7044909275078|
|[31.7656188210424...| 501.8648884818972|
|[31.8164283341993...| 519.9879294056627|
|[31.8279790554652...|449.86441308254825|
|[31.8745516945853...|398.35142503580505|
|[31.9120759292006...| 389.8997824847104|
|[31.9549038566348...| 432.0730114878495|
|[32.0215955013870...| 517.8183001311386|
|[32.0444861274404...| 447.9547934371519|
|[32.0478009788678...|508.85062863952567|
+--------------------+------------